<a href="https://colab.research.google.com/github/squinton-gcu/Data-Science/blob/main/Processing_Module_R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# check and install all packages if needed
# if you do need to download anything, please restart the runtime
if("googledrive" %in% rownames(installed.packages()) == FALSE) {install.packages("googledrive")}
if("BiocManager" %in% rownames(installed.packages()) == FALSE) {
  install.packages("BiocManager")
  if("MAI" %in% rownames(installed.packages()) == FALSE) {BiocManager::install("MAI")} 
  # issue with rlang during instalation which requires it to be re-downloaded
   remove.packages('rlang')
  install.packages('rlang')
   }
if("enetLTS" %in% rownames(installed.packages()) == FALSE) {install.packages("enetLTS")}

In [3]:
library('googledrive')
library('MAI')
library('enetLTS')

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
Registered S3 method overwritten by 'cvTools':
  method        from 
  print.cvFolds perry



In [4]:
if (file.exists("/usr/local/lib/python3.7/dist-packages/google/colab/_ipython.py")) { #may update python version  
                                       #occasionally
  install.packages("R.utils")
  library("R.utils")
  library("httr")
  my_check <- function() {return(TRUE)}
  reassignInPackage("is_interactive", pkgName = "httr", my_check) 
  options(rlang_interactive=TRUE)
}

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘R.oo’, ‘R.methodsS3’


Loading required package: R.oo

Loading required package: R.methodsS3

R.methodsS3 v1.8.2 (2022-06-13 22:00:14 UTC) successfully loaded. See ?R.methodsS3 for help.

R.oo v1.25.0 (2022-06-12 02:20:02 UTC) successfully loaded. See ?R.oo for help.


Attaching package: ‘R.oo’


The following object is masked from ‘package:R.methodsS3’:

    throw


The following objects are masked from ‘package:methods’:

    getClasses, getMethods


The following objects are masked from ‘package:base’:

    attach, detach, load, save


R.utils v2.12.0 (2022-06-28 03:20:05 UTC) successfully loaded. See ?R.utils for help.


Attaching package: ‘R.utils’


The following object is masked from ‘package:utils’:

    timestamp


The following objects are masked from ‘package:base’:

    cat, commandArgs, getOption, isOpen, nullfile, parse, warnings




In [6]:
drive_auth(use_oob = TRUE, cache = FALSE)

Please point your browser to the following url: 

https://accounts.google.com/o/oauth2/auth?client_id=603366585132-dpeg5tt0et3go5of2374d83ifevk5086.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code



Enter authorization code: 4/1AX4XfWjXHub-OnpG436JokY3jsXESP0G8HGr-0cUwnY--LYOPW2im_LsjdQ


In [8]:
x <- drive_get('~/Thesis/ALZ_plasma.csv')
drive_download(x, overwrite = TRUE)
y <- drive_get('~/Thesis/ALZ_CSF2.csv')
drive_download(y, overwrite = TRUE)
z <- drive_get('~/Thesis/trauma_human.csv')
drive_download(z, overwrite = TRUE)
v <- drive_get('~/Thesis/rat_stress.csv')
drive_download(v, overwrite = TRUE)





✔ The input `path` resolved to exactly 1 file.

File downloaded:

• ALZ_plasma.csv <id: 1U7njj5PBdw9bSX0J_e-Ri88Vfz2kQf87>

Saved locally as:

• ALZ_plasma.csv



In [9]:
ALZ_plasma <- read.table("ALZ_plasma.csv", header=TRUE, row.names = 1, sep =',')
ALZ_CSF <- read.table("ALZ_CSF2.csv", header=TRUE, row.names=1, sep=',')
trauma_human <- read.table("trauma_human.csv", header=TRUE, row.names=1, sep=',')
trauma_rat <- read.table("rat_stress.csv", header=TRUE, row.names=1, sep=',')

In [27]:
processing_step_impute <- function(Input_File) {
  #remove all metabolites with more than 70% missing values
  Input_File2 <- Input_File[, which(colMeans(!is.na(Input_File)) > 0.7)]
  # take care of missing values
  imputed_File <- MAI(Input_File2, 
					MCAR_algorithm = c("Multi_nsKNN"),
					MNAR_algorithm = c("nsKNN"),
					)
  imputed_File <- as.data.frame(imputed_File)
  row.names(imputed_File) = row.names(Input_File2)
  colnames(imputed_File) = colnames(Input_File2)
  if (ncol(Input_File2) != ncol(imputed_File)) {
    imputed_File2 <- imputed_File[1:(length(imputed_File)-(ncol(imputed_File) - ncol(Input_File2)))]
  }
  else {
    imputed_File2 = imputed_File
  }
  colnames(imputed_File2) = colnames(Input_File2)

  return(imputed_File2)
}

processing_step_normalize <- function(imputedFile) {
  # normalize with log2 and change NAs to zeros
  normalized_File <- as.data.frame(log2(imputedFile))
  normalized_File[is.na(normalized_File)] <- 0
  return(normalized_File)
}

processing_step_scale <- function(normalizedFile) {
  # Scale using the z scale function from R
  scale_File <- scale(t(normalizedFile))
  scale_File <- t(scale_File)
  return(scale_File)
}


In [25]:
# will generate a few random initial exploratory graphs to ensure processing was successful
graph_normalizeVSscale <- function(normalized_table, scale_table, table_name) {
  png(paste(table_name, "_scale1.png"))
  scale1_hist <- hist(as.numeric(scale_table[1,]), main = paste("Histogram of " , table_name, "scale1"))
  dev.off()
  png(paste(table_name, "_scale12.png"))
  scale12_hist <- hist(as.numeric(scale_table[15,]), main = paste("Histogram of " , table_name, "scale2"))
  dev.off()

  png(paste(table_name, "_normalized1.png"))
  normalized1_hist <- hist(as.numeric(normalized_table[1,]), main = paste("Histogram of " , table_name, "normalized1"))
  dev.off()
  png(paste(table_name, "_normalized2.png"))
  normalized2_hist <- hist(as.numeric(normalized_table[15,]), main = paste("Histogram of " , table_name, "normalized2"))
  dev.off()
}

In [28]:
ALZ_plasma_imputed <- processing_step_impute(ALZ_plasma)
ALZ_plasma_normalized <- processing_step_normalize(ALZ_plasma_imputed)
ALZ_plasma_scaled <- processing_step_scale(ALZ_plasma_normalized)

Warning message in MAI(Input_File2, MCAR_algorithm = c("Multi_nsKNN"), MNAR_algorithm = c("nsKNN"), :
“The smallest data set MAI has been tested on was a 50x50 matrix.
            Accuracy can not be guaranteed for smaller data sets.”
Estimating pattern of missingness

Imposing missingness

Generating features

Training

Predicting

Imputing

Warning message in lapply(X = x, FUN = .Generic, ...):
“NaNs produced”
Warning message in lapply(X = x, FUN = .Generic, ...):
“NaNs produced”
Warning message in lapply(X = x, FUN = .Generic, ...):
“NaNs produced”
Warning message in lapply(X = x, FUN = .Generic, ...):
“NaNs produced”
Warning message in lapply(X = x, FUN = .Generic, ...):
“NaNs produced”
Warning message in lapply(X = x, FUN = .Generic, ...):
“NaNs produced”
Warning message in lapply(X = x, FUN = .Generic, ...):
“NaNs produced”
Warning message in lapply(X = x, FUN = .Generic, ...):
“NaNs produced”
Warning message in lapply(X = x, FUN = .Generic, ...):
“NaNs produced”
Warning message 

In [ ]:
ALZ_csf_imputed <- processing_step_impute(ALZ_CSF)
ALZ_csf_normalized <- processing_step_normalize(ALZ_csf_imputed)
ALZ_csf_scaled <- processing_step_scale(ALZ_csf_normalized)

In [ ]:
trauma_human_imputed <- processing_step_impute(trauma_human)
trauma_human_normalized <- processing_step_normalize(trauma_human_imputed)
trauma_human_scaled <- processing_step_scale(trauma_human_normalized)

In [ ]:
trauma_rat_imputed <- processing_step_impute(trauma_rat)
trauma_rat_normalized <- processing_step_normalize(trauma_rat_imputed)
trauma_rat_scaled <- processing_step_scale(trauma_rat_normalized)

In [20]:
graph_normalizeVSscale(ALZ_plasma_normalized, ALZ_plasma_scaled, "ALZ_plasma")
graph_normalizeVSscale(ALZ_csf_normalized, ALZ_csf_scaled, "ALZ_csf")
graph_normalizeVSscale(trauma_human_normalized, trauma_human_scaled, "trauma_human")
graph_normalizeVSscale(trauma_rat_normalized, trauma_rat_scaled, "trauma_rat")

png 
  2